https://pbpython.com/amortization-model.html

import libraries and modules

Things learned from this exercise
    datetime needs to be imported as datetime for full functionallity to work for this project
    input defaults as a string, but you can change it by surrounding the input function in float or int to change to numbers
    strptime() takes the date string as a variable for the first argument, and the format '%Y-%m-%d' as the second argument and returns a value in date format
    pd.date_range() function takes the start date (either as a variable or value), the length of the date range/how many months (periods), and the frequency (MS stands for month/start)
    then we created a dataframe with the index range equal to the date_range, as well as labeling the columns we are will calculate
    we want to reset the index so the first payment period starts at 1 instead of 0, so we use df.reset_index(inplace=True) and on the next line we set the df.index to +=1 (it will add 1 to the current index), finally we rename the index column
    calculate the Payment column using the monthly payment formula (P * (i / (1 - (1 + i) ** (-1 * (n))))) where P = principal borrowed, i = monthly interest rate (interest rate / 12), and n = number of payments (length of loan * 12)
    np.ppmt(rate, per, nper) calculates monthly principal using the monthly interest for the first argument, per is the period of the interest (as a numeric value, which is why the df.index is used), and nper is the number of compounding interest
    np.ipmt(rate, per, nper): same as np.ppmt()
    new columns can be calculated by declaring the df['col_name'] = (calculation)
    [Cumulative_Principal'] takes the value in the calculated ['Principal'] column and adds the ['Addl_Payment'] column, and adds the sum from the previous index (.cumsum()) for a running total
    ['Cumulative_Interest'] same as ['Cumulative_Principal']
    ['Balance'] is calculated be subtracting the total Principal borrowed by the ['Cumulative_Principal'] paid. If there are additional payments, values are set to 0 for negative 'payments'
    the dataframe is truncated automatically by keeping all rows that are non zero in the balance column
    finally, the entire dataframe is rounded to 2 decimal places
    pd.set_options() allows the entire datatable to be visible

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

define variables needed

In [2]:
Interest_Rate = float(input("Enter your interest rate: "))
Monthly_interest = Interest_Rate / 12 / 100

In [3]:
Years = float(input("Enter the legnth of loan in years: "))
Payments_Year = 12

In [4]:
Principal = float(input("Enter your loan amount in $: "))

In [5]:
Principal_paydown = float(input("Enter your paydown amount (down payment for auto loans): "))

In [6]:
Addl_Payment = float(input("Enter additional monthly payments (if any): "))

In [7]:
start_date_input = str(input("Enter the start date as yyyy-mm-dd: "))
start_date = datetime.strptime(start_date_input, "%Y-%m-%d")

This helpful function creates a range for the next 30 years starting on Jan 1, 2016. The range will be used to build up the basic DataFrame we will use for the amortization schedule. Note that we need to make sure the first period is 1 not 0, hence the need to use the df.index += 1 :

build a datetimeindex for the next x amount of years based on MS (month, start)

In [8]:
date_range = pd.date_range(start_date, periods=(Years * Payments_Year), freq='MS')
date_range.name = 'Payment_Date'

# periods: number of periods to generate, freq: "MS" stands for "month start"

print(date_range)

DatetimeIndex(['2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01',
               '2021-05-01', '2021-06-01', '2021-07-01', '2021-08-01',
               '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01',
               '2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01',
               '2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01',
               '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01',
               '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01',
               '2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01',
               '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01'],
              dtype='datetime64[ns]', name='Payment_Date', freq='MS')


In [9]:
df = pd.DataFrame(index=date_range,columns=['Payment', 'Principal', 'Interest', 'Addl_Payment', 'Balance'], dtype='float')
df.reset_index(inplace=True)
df.index += 1
df.index.name = "Payment Number"

print(df)

               Payment_Date  Payment  Principal  Interest  Addl_Payment  \
Payment Number                                                            
1                2021-01-01      NaN        NaN       NaN           NaN   
2                2021-02-01      NaN        NaN       NaN           NaN   
3                2021-03-01      NaN        NaN       NaN           NaN   
4                2021-04-01      NaN        NaN       NaN           NaN   
5                2021-05-01      NaN        NaN       NaN           NaN   
6                2021-06-01      NaN        NaN       NaN           NaN   
7                2021-07-01      NaN        NaN       NaN           NaN   
8                2021-08-01      NaN        NaN       NaN           NaN   
9                2021-09-01      NaN        NaN       NaN           NaN   
10               2021-10-01      NaN        NaN       NaN           NaN   
11               2021-11-01      NaN        NaN       NaN           NaN   
12               2021-12-

calculate the payment, Principal, Interest, Addl_Principal columns

In [13]:
df['Payment'] = (Principal - Principal_paydown) * (Monthly_interest / (1 - (1 + Monthly_interest)**(-(Years*Payments_Year))))

df["Principal"] = -1 * np.ppmt(Monthly_interest, df.index, Years*Payments_Year, (Principal - Principal_paydown))

df["Interest"] = -1 * np.ipmt(Monthly_interest, df.index, Years*Payments_Year, (Principal - Principal_paydown))

df["Addl_Payment"] = Addl_Payment

df['Cumulative_Principal'] = (df['Principal'] + df['Addl_Payment']).cumsum()

df['Cumulative_Interest'] = (df['Interest']).cumsum()

df['Balance'] = (Principal - Principal_paydown) - df['Cumulative_Principal']

df['Balance'] = df['Balance'].clip(lower=0)
df = df[df['Balance']!=0]

df = df.round(2)

pd.set_option("display.max_rows", None, "display.max_columns", None)
print(df)

               Payment_Date  Payment  Principal  Interest  Addl_Payment  \
Payment Number                                                            
1                2021-01-01    73.26      65.96      7.29           0.0   
2                2021-02-01    73.26      66.16      7.10           0.0   
3                2021-03-01    73.26      66.35      6.91           0.0   
4                2021-04-01    73.26      66.54      6.71           0.0   
5                2021-05-01    73.26      66.74      6.52           0.0   
6                2021-06-01    73.26      66.93      6.32           0.0   
7                2021-07-01    73.26      67.13      6.13           0.0   
8                2021-08-01    73.26      67.32      5.93           0.0   
9                2021-09-01    73.26      67.52      5.74           0.0   
10               2021-10-01    73.26      67.72      5.54           0.0   
11               2021-11-01    73.26      67.91      5.34           0.0   
12               2021-12-